# Upload game data
Natalia Vélez, April 2021

This notebook reads object, category, and transition data contained within the [OneLifeData7](https://github.com/jasonrohrer/OneLifeData7) repository and uploads game data to the database.

In [1]:
import pymongo
import os
import fnmatch
import json
import pprint
import pandas as pd
import numpy as np
from tqdm import notebook

# Project-specific modules
import sys
sys.path.append('..')
from utils import gsearch, int_extract
import ohol_objects as obj
import ohol_categories as cat
import ohol_transitions as trans

data_dir = '../../OneLifeData7'

## Process game data

### Objects

Get all object files:

In [2]:
obj_files = gsearch(data_dir, 'objects', '[0-9]*.txt')
obj_files.sort()
print('Found %i objects' % len(obj_files))
print(*obj_files[:10], sep='\n')
print('...')

Found 4161 objects
../../OneLifeData7/objects/100.txt
../../OneLifeData7/objects/1000.txt
../../OneLifeData7/objects/1001.txt
../../OneLifeData7/objects/1002.txt
../../OneLifeData7/objects/1003.txt
../../OneLifeData7/objects/1006.txt
../../OneLifeData7/objects/1007.txt
../../OneLifeData7/objects/1008.txt
../../OneLifeData7/objects/1009.txt
../../OneLifeData7/objects/1010.txt
...


Helper function: Read object data

In [3]:
def load_obj(path):
    obj_id = int_extract('(?<=objects/)[0-9]+(?=\.txt)', path)
    obj_data = obj.read_obj(obj_id)
    return obj_data

print(load_obj(obj_files[0]))

{'id': 100, 'name': 'White Pine Tree with Needles', 'containable': 0, 'containSize': 1, 'vertSlotRot': 0.0, 'permanent': 1, 'minPickupAge': 3, 'heldInHand': 0, 'blocksWalking': 1, 'leftBlockingRadius': 0, 'rightBlockingRadius': 0, 'drawBehindPlayer': 0, 'mapChance': 1.0, 'heatValue': 0, 'rValue': 0.0, 'person': 0, 'noSpawn': 0, 'male': 0, 'deathMarker': 0, 'foodValue': 0, 'speedMult': 1.0, 'heldOffset': [0.0, 0.0], 'clothing': 'n', 'clothingOffset': [0.0, 0.0], 'deadlyDistance': 0, 'useDistance': 1, 'sounds': ['-1:0.250000', '-1:0.250000', '-1:0.250000', '-1:1.000000'], 'creationSoundInitialOnly': 0, 'numSlots': 0, 'timeStretch': 1.0, 'slotSize': 1, 'numSprites': 7, 'spriteID': 317, 'pos': [-3.0, 0.0], 'rot': 0.0, 'hFlip': 0, 'color': [1.0, 1.0, 1.0], 'ageRange': [-1.0, -1.0], 'parent': -1, 'invisHolding': 0, 'invisWorn': 0, 'behindSlots': 0, 'headIndex': -1, 'bodyIndex': -1, 'backFootIndex': -1, 'frontFootIndex': -1, 'numUses': 1, 'useVanishIndex': -1, 'useAppearIndex': -1, 'pixHeight

Iterate over all objects:

In [4]:
# Load data for all objects
all_objects = []
for f in notebook.tqdm(obj_files):
    all_objects.append(load_obj(f))

# Sort objects in order added to game
all_objects.sort(key=lambda o:o['id'])

  0%|          | 0/4161 [00:00<?, ?it/s]

Example object:

In [5]:
pprint.pp(all_objects[10])

{'id': 40,
 'name': 'Wild Carrot',
 'containable': 1,
 'containSize': 1,
 'vertSlotRot': 0.0,
 'permanent': 0,
 'minPickupAge': 3,
 'heldInHand': 1,
 'blocksWalking': 0,
 'leftBlockingRadius': 0,
 'rightBlockingRadius': 0,
 'drawBehindPlayer': 0,
 'mapChance': 0.0,
 'heatValue': 0,
 'rValue': 0.0,
 'person': 0,
 'noSpawn': 0,
 'male': 0,
 'deathMarker': 0,
 'foodValue': 5,
 'speedMult': 1.0,
 'heldOffset': [1.0, -3.0],
 'clothing': 'n',
 'clothingOffset': [0.0, 0.0],
 'deadlyDistance': 0,
 'useDistance': 1,
 'sounds': ['-1:0.250000', '-1:0.250000', '143:0.250000', '-1:1.000000'],
 'creationSoundInitialOnly': 0,
 'numSlots': 0,
 'timeStretch': 1.0,
 'slotSize': 1,
 'numSprites': 1,
 'spriteID': 163,
 'pos': [-3.0, -40.0],
 'rot': -0.25,
 'hFlip': 0,
 'color': [1.0, 1.0, 1.0],
 'ageRange': [-1.0, -1.0],
 'parent': -1,
 'invisHolding': 0,
 'invisWorn': 0,
 'behindSlots': 0,
 'headIndex': -1,
 'bodyIndex': -1,
 'backFootIndex': -1,
 'frontFootIndex': -1,
 'numUses': 1,
 'useVanishIndex': -

### Categories

Get all category files:

In [6]:
cat_files = gsearch(data_dir, 'categories', '*.txt')
cat_files.sort()
print('%i categories found' % len(cat_files))
print(*cat_files[:10], sep='\n')
print('...')

269 categories found
../../OneLifeData7/categories/1001.txt
../../OneLifeData7/categories/1016.txt
../../OneLifeData7/categories/1017.txt
../../OneLifeData7/categories/1018.txt
../../OneLifeData7/categories/1019.txt
../../OneLifeData7/categories/1022.txt
../../OneLifeData7/categories/1023.txt
../../OneLifeData7/categories/1024.txt
../../OneLifeData7/categories/1033.txt
../../OneLifeData7/categories/1034.txt
...


Helper function: Read category data

In [7]:
def load_cat(path):
    cat_id = int_extract('(?<=categories/)[0-9]+(?=\.txt)', path)
    cat_data = cat.read_cat(cat_id)
    return cat_data

print(load_cat(cat_files[0]))

{'parentID': 1001, 'numObjects': 2, 'probabilistic': False, 'pattern': False, 'children': [912, 1000], 'probs': None}


Load category data:

In [8]:
all_cats = []
for f in notebook.tqdm(cat_files):
    all_cats.append(load_cat(f))
    
# Sort categories by ID
all_cats.sort(key=lambda c:c['parentID'])

  0%|          | 0/269 [00:00<?, ?it/s]

In [9]:
pprint.pp(all_cats[0])

{'parentID': 316,
 'numObjects': 3,
 'probabilistic': False,
 'pattern': True,
 'children': [2505, 2517, 3002],
 'probs': None}


### Transitions

Find transition files:

In [10]:
trans_files = gsearch(data_dir, 'transitions', '*.txt')
print('Found %i transitions' % len(trans_files))
print(*trans_files[:10], sep='\n')

Found 4847 transitions
../../OneLifeData7/transitions/67_2970.txt
../../OneLifeData7/transitions/235_1890.txt
../../OneLifeData7/transitions/441_445.txt
../../OneLifeData7/transitions/0_2671.txt
../../OneLifeData7/transitions/239_335.txt
../../OneLifeData7/transitions/405_254.txt
../../OneLifeData7/transitions/0_4302.txt
../../OneLifeData7/transitions/0_2359.txt
../../OneLifeData7/transitions/-1_420_LT.txt
../../OneLifeData7/transitions/4542_-1.txt


In [11]:
trans.read_transition('../../OneLifeData7/transitions/-1_420_LT.txt')

{'origActor': -1,
 'origTarget': 420,
 'newActor': 0,
 'newTarget': 429,
 'autoDecaySeconds': 1,
 'actorMinUseFraction': 0.0,
 'targetMinUseFraction': 0.0,
 'reverseUseActor': 0,
 'reverseUseTarget': 0,
 'move': 2,
 'desiredMoveDist': 1,
 'lastUseActor': False,
 'lastUseTarget': True,
 'origActorName': 'Empty',
 'origTargetName': 'Shot Wolf',
 'newActorName': 'Empty',
 'newTargetName': 'Dying Shot Wolf',
 'isTool': False}

In [12]:
all_transitions = []
for f in notebook.tqdm(trans_files):
    all_transitions.append(trans.read_transition(f))

  0%|          | 0/4847 [00:00<?, ?it/s]

In [13]:
pprint.pp(all_transitions[0])

{'origActor': 67,
 'origTarget': 2970,
 'newActor': 0,
 'newTarget': 2966,
 'autoDecaySeconds': 0,
 'actorMinUseFraction': 0.0,
 'targetMinUseFraction': 0.0,
 'reverseUseActor': 0,
 'reverseUseTarget': 0,
 'move': 0,
 'desiredMoveDist': 1,
 'noUseActor': 0,
 'noUseTarget': 0,
 'lastUseActor': False,
 'lastUseTarget': False,
 'origActorName': 'Long Straight Shaft',
 'origTargetName': 'Approved Property Fence - +horizontalB +wall',
 'newActorName': 'Empty',
 'newTargetName': 'Proposed Property Gate - +causeAutoOrientH +wall',
 'isTool': True}


## Upload to database

### Connect to database

In [14]:
#find *.key containing username and password on separate lines(untracked using .gitignore)
for file_name in os.listdir():
    if fnmatch.fnmatch(file_name,'*.key'):
        keyfile = file_name

In [15]:
#Connection string
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
print(myclient)

MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol')


In [16]:
ohol = myclient.ohol
ohol.list_collection_names()

['lifelogs', 'svd', 'families', 'test', 'fs.files', 'jobmatrix', 'fs.chunks']

Insert objects:

In [17]:
obj_col = ohol.objects
obj_col.insert_many(all_objects)

Insert categories:

In [20]:
cat_col = ohol.categories
cat_col.insert_many(all_cats)

Insert transitions:

In [21]:
trans_col = ohol.transitions
trans_col.insert_many(all_transitions)

In [22]:
ohol.list_collection_names()

['lifelogs',
 'svd',
 'objects',
 'transitions',
 'families',
 'test',
 'fs.files',
 'jobmatrix',
 'fs.chunks',
 'categories']